In [254]:
import pandas as pd
import numpy as np

import re
import string

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

import spacy
nlp = spacy.load("en_core_web_sm")

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/raghavsharma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/raghavsharma/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/raghavsharma/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [255]:
temp_df = pd.read_csv('../../data/temp_data.csv')
temp_df.head()

,time,comment,event,event_player,event_team,comment_desc
0,NaN,We hope you have enjoyed our live coverage of...,NaN,NaN,NaN,full time summary
1,NaN,While Madrid's Champions League defence conti...,NaN,NaN,NaN,full time summary
2,NaN,"It's all over at Stamford Bridge, and it's an...",NaN,NaN,NaN,full time summary
3,90 + 3,FULL-TIME: CHELSEA 0-2 REAL MADRID,NaN,NaN,NaN,timer
4,90 + 3,"Valverde sweeps a pass out to Ceballos, who c...",NaN,NaN,NaN,timer


# Cleaning the data

In [256]:
def remove_punct(text):
    """ A method to remove punctuations from text """
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text) #removes numbers from text
    return text

In [257]:
def remove_stopwords(text):
    """ A method to remove all the stopwords """
    stopwords = nltk.corpus.stopwords.words('english')
    text = [word for word in text if word not in stopwords]
    return text

In [258]:
def tokenization(text):
    """ A method to tokenize text data """
    text = re.split('\W+', text) #splitting each sentence/ tweet into its individual words
    return text

In [259]:
def stemming(text):
    """ A method to perform stemming on text data"""
    porter_stem = nltk.PorterStemmer()
    text = [porter_stem.stem(word) for word in text]
    return text

In [260]:
def lemmatizer(text):
    word_net_lemma = nltk.WordNetLemmatizer()
    text = [word_net_lemma.lemmatize(word) for word in text]
    return text

In [261]:
# Making a common cleaning function for every part below for code reproducability
def clean_text(list_words):
    # Making a regex pattern to match in the characters we would like to replace from the words
    character_replace = ",()0123456789.?!@#$%&;*:_,/" 
    pattern = "[" + character_replace + "]"
    # ------------------------------------------------------------------------------------

    # ------------------------------------------------------------------------------------
    new_list_words = []
    # Looping through every word to remove the characters and appending back to a new list
    # replace is being used for the characters that could not be catched through regex
    for s in list_words:
        new_word = s.lower()
        new_word = re.sub(pattern,"",new_word)
        new_word = new_word.replace('[', '')
        new_word = new_word.replace(']', '')
        new_word = new_word.replace('-', '')
        new_word = new_word.replace('—', '')
        new_word = new_word.replace('“', '')
        new_word = new_word.replace("’", '')
        new_word = new_word.replace("”", '')
        new_word = new_word.replace("‘", '')
        new_word = new_word.replace('"', '')
        new_word = new_word.replace("'", '')
        new_word = new_word.replace(" ", '')
        new_list_words.append(new_word)

    # Using filter to remove empty strings
    new_list_words = list(filter(None, new_list_words))
    return new_list_words

In [262]:
def clean(df, text_col):
    """ A method to do basic data cleaning """
    
    clean_data = df.copy()
    
    clean_data['clean_text']=clean_data[text_col].apply(lambda x: remove_punct(x))
    
    clean_data['text_tokenized'] = clean_data['clean_text'].apply(lambda x: tokenization(x.lower()))
    
    stopwords = nltk.corpus.stopwords.words('english')
    
    clean_data['text_without_stop'] = clean_data['text_tokenized'].apply(lambda x: remove_stopwords(x))    
    
    clean_data['text_stemmed'] = clean_data['text_without_stop'].apply(lambda x: stemming(x))
        
    clean_data['text_lemmatized'] = clean_data['text_without_stop'].apply(lambda x: lemmatizer(x))

    clean_data['text_final'] = clean_data['text_lemmatized'].apply(lambda x: clean_text(x))
    
    return clean_data

In [263]:
def clean(corpus):
    """ A method to do basic data cleaning """
    clean_data = pd.DataFrame(columns=['ini_text', 'clean_text', 'text_tokenized', 'text_without_stop', 
                                       'text_stemmed', 'text_lemmatized', 'text_final'])

    clean_data['ini_text'] = [corpus]

    clean_data['clean_text']=clean_data['ini_text'].apply(lambda x: remove_punct(x))
    
    clean_data['text_tokenized'] = clean_data['clean_text'].apply(lambda x: tokenization(x.lower()))
    
    stopwords = set(nltk.corpus.stopwords.words('english'))
    
    clean_data['text_without_stop'] = clean_data['text_tokenized'].apply(lambda x: remove_stopwords(x))    
    
    clean_data['text_stemmed'] = clean_data['text_without_stop'].apply(lambda x: stemming(x))
        
    clean_data['text_lemmatized'] = clean_data['text_without_stop'].apply(lambda x: lemmatizer(x))

    clean_data['text_final'] = clean_data['text_lemmatized'].apply(lambda x: clean_text(x))
    
    return clean_data

# Using SpaCy to summarize the text

In [264]:
def summarize_text(text, num_sentences=3):
    doc = nlp(text)
    sentences = [sent for sent in doc.sents]
    sentence_scores = {}
    for i, sent in enumerate(sentences):
        sentence_scores[i] = sent.similarity(doc)
    top_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]
    summary = [sentences[i].text.strip() for i in top_sentences]
    return " ".join(summary)

In [265]:
df1 = pd.DataFrame(columns=['comment'])
df1['comment'] = temp_df[temp_df['comment_desc'] == 'timer']['comment']
df1

,comment
3,FULL-TIME: CHELSEA 0-2 REAL MADRID
4,"Valverde sweeps a pass out to Ceballos, who c..."
5,Camavinga and Valverde team up to deny Sterli...
6,We're into three minutes of stoppage time as ...
7,Mudryk picks up a late booking for a frustrat...
...,...
82,Havertz is playing a lone striking role for C...
83,"Kante steals possession from Benzema, but Mad..."
84,Cucurella is on hand to intercept a slack bal...
85,Havertz looks to get Chelsea on the front foo...


In [266]:
comment = ''
for comm in df1['comment']:
    comment += comm

spacy_summary = summarize_text(comment)
print(spacy_summary)

Vinicius gets the assist for Rodrygo's goal, staying cool under pressure at the far post to pick out his opposite winger, who makes no mistake from close range!  GOAAAAAAAL! James works some space to hit a low cross into the six-yard box, but Alaba slides in to divert it away from the danger zone! A rare loose ball from Modric gives Chelsea the chance to put Madrid under pressure, but Kante is crowded out near the edge of the area and Madrid survive.


# Using nltk for text summarization

In [267]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
from heapq import nlargest

def summarize_text_nltk(text, num_sentences=3):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)
    
    # Tokenize the sentences into words and remove stopwords

    # stop_words = set(stopwords.words("english"))
    
    text_punct_removed = remove_punct(text)
    words = tokenization(text_punct_removed.lower())
    
    # words = word_tokenize(text)

    # remove stopwords
    stop_words =  set(nltk.corpus.stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    
    # Apply stemming to the filtered words
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in filtered_words]

    # Lemmatize words
    word_net_lemma = nltk.WordNetLemmatizer()
    word_lemma = [word_net_lemma.lemmatize(word) for word in stemmed_words]
    
    # Calculate word frequency and sentence scores
    word_freq = nltk.FreqDist(stemmed_words)
    sentence_scores = {}
    for i, sentence in enumerate(sentences):
        for word in nltk.word_tokenize(sentence.lower()):
            if word in word_freq:
                if len(sentence.split()) < 30:
                    if i not in sentence_scores:
                        sentence_scores[i] = word_freq[word]
                    else:
                        sentence_scores[i] += word_freq[word]
    
    # Select the top sentences based on their scores
    summary_sentences = nlargest(num_sentences, sentence_scores, key=sentence_scores.get)
    summary = [sentences[i] for i in sorted(summary_sentences)]
    return " ".join(summary)

In [268]:
comment = ''
for comm in df1['comment']:
    comment += comm

nltk_summary = summarize_text_nltk(comment)
print(nltk_summary)

 FULL-TIME: CHELSEA 0-2 REAL MADRID  Valverde sweeps a pass out to Ceballos, who cuts inside and shoots straight at Kepa from the edge of the area. A rare loose ball from Modric gives Chelsea the chance to put Madrid under pressure, but Kante is crowded out near the edge of the area and Madrid survive. Cucurella is on hand to intercept a slack ball from Valverde in midfield, and Chelsea can look to build an attack.


# Using GPT-3

In [310]:
comment

" FULL-TIME: CHELSEA 0-2 REAL MADRID   Valverde sweeps a pass out to Ceballos, who cuts inside and shoots straight at Kepa from the edge of the area. That should be that for Madrid, and for Chelsea's Champions League campaign.   Camavinga and Valverde team up to deny Sterling a route to the byline. Though Chelsea missed a couple of clear chances at 0-0, Madrid's backline has stood firm for the majority of this game.   We're into three minutes of stoppage time as Felix shoots well over. Real Madrid's fans are in great voice, their defence of the Champions League trophy is set to continue! For Chelsea, a trophyless campaign is about to become reality.   Mudryk picks up a late booking for a frustrated lunge on Modric in midfield.   Now Mudryk bursts through on goal at the other end and smashes into the side-netting, only for the flag to go up. The Ukrainian mistimed his run, but his wayward finish was typical of Chelsea's efforts in the final third.   Good save, but the flag is up! Vinici

# Using Gensim

In [304]:
!pip install gensim==3.8.3

In [305]:
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
import wikipedia
import en_core_web_sm

# Get wiki content.
wikisearch = wikipedia.page("Amitabh Bachchan")
wikicontent = wikisearch.content
nlp = en_core_web_sm.load()
doc = nlp(wikicontent)

# Save the wiki content to a file
# (for reference).
f = open("wikicontent.txt", "w")
f.write(wikicontent)
f.close()

# Summary (0.5% of the original content).
summ_per = summarize(wikicontent, ratio = 0.05)
print("Percent summary")
print(summ_per)

# Summary (200 words)
summ_words = summarize(wikicontent, word_count = 200)
print("Word count summary")
print(summ_words)

ModuleNotFoundError: No module named 'gensim.summarization'

# Using sliding window technique

In [269]:
def window_df(df, start_timer, end_timer):
    comments = []
    for i in range(len(df)):
        time = df['time'][i]
        if time != 'nan':
            if '+' in time:
                time = time[:2]
            if int(time) >= start_timer and int(time) < end_timer:
                if df['comment_desc'][i] == 'timer':
                    comments.append(df['comment'][i])
    return " ".join(comments)

In [270]:
# Convert time column to str
temp_df['time'] = temp_df['time'].astype(str)

# Divide the dataframe into 6 separate dfs, each corresponding to 15 minutes of the match.
comm_15 = window_df(temp_df, 0, 16)
comm_30 = window_df(temp_df, 16, 31)
comm_45 = window_df(temp_df, 31, 46)
comm_60 = window_df(temp_df, 46, 61)
comm_75 = window_df(temp_df, 61, 76)
comm_90 = window_df(temp_df, 76, 91)

comments = [comm_15, comm_30, comm_45, comm_60, comm_75, comm_90]

In [271]:
summaries = []
for comment  in comments:
    summaries.append(summarize_text_nltk(comment))

summaries

['A rare loose ball from Modric gives Chelsea the chance to put Madrid under pressure, but Kante is crowded out near the edge of the area and Madrid survive. Cucurella is on hand to intercept a slack ball from Valverde in midfield, and Chelsea can look to build an attack. Havertz looks to get Chelsea on the front foot straight away, winning a throw-in high on the left.',
 "Lampard's side have a spare man inside the Madrid area as Cucurella breaks down the left, but his delivery is overhit and it bypasses everyone. A flowing Madrid move involving Benzema and Rodrygo sees Vinicius released into the left-hand channel, and the Brazilian cuts inside to shoot straight at Kepa. The Madrid man is booked as Chelsea get a free-kick in a dangerous position, with Fernandez set to deliver.",
 " HALF-TIME: CHELSEA 0-0 REAL MADRID   HUGE SAVE FROM COURTOIS! So much of Chelsea's attacking play has come down that flank, but wing-back James appeared to be limping a moment ago, which will be a big concer

In [272]:
timer_list= ['[1-15]', '[16-30]', '[31-45]', '[46-60]', '[61-75]', '[76-90]']

In [273]:
summaries_time = []
for i in range(len(summaries)):
    summaries_time.append(timer_list[i] + ' ' +  summaries[i])

summaries_time

['[1-15] A rare loose ball from Modric gives Chelsea the chance to put Madrid under pressure, but Kante is crowded out near the edge of the area and Madrid survive. Cucurella is on hand to intercept a slack ball from Valverde in midfield, and Chelsea can look to build an attack. Havertz looks to get Chelsea on the front foot straight away, winning a throw-in high on the left.',
 "[16-30] Lampard's side have a spare man inside the Madrid area as Cucurella breaks down the left, but his delivery is overhit and it bypasses everyone. A flowing Madrid move involving Benzema and Rodrygo sees Vinicius released into the left-hand channel, and the Brazilian cuts inside to shoot straight at Kepa. The Madrid man is booked as Chelsea get a free-kick in a dangerous position, with Fernandez set to deliver.",
 "[31-45]  HALF-TIME: CHELSEA 0-0 REAL MADRID   HUGE SAVE FROM COURTOIS! So much of Chelsea's attacking play has come down that flank, but wing-back James appeared to be limping a moment ago, whi

In [274]:
summ_comm = " ".join(summaries_time)
summ_comm

"[1-15] A rare loose ball from Modric gives Chelsea the chance to put Madrid under pressure, but Kante is crowded out near the edge of the area and Madrid survive. Cucurella is on hand to intercept a slack ball from Valverde in midfield, and Chelsea can look to build an attack. Havertz looks to get Chelsea on the front foot straight away, winning a throw-in high on the left. [16-30] Lampard's side have a spare man inside the Madrid area as Cucurella breaks down the left, but his delivery is overhit and it bypasses everyone. A flowing Madrid move involving Benzema and Rodrygo sees Vinicius released into the left-hand channel, and the Brazilian cuts inside to shoot straight at Kepa. The Madrid man is booked as Chelsea get a free-kick in a dangerous position, with Fernandez set to deliver. [31-45]  HALF-TIME: CHELSEA 0-0 REAL MADRID   HUGE SAVE FROM COURTOIS! So much of Chelsea's attacking play has come down that flank, but wing-back James appeared to be limping a moment ago, which will b

In [275]:
print(summarize_text_nltk(summ_comm))

Cucurella is on hand to intercept a slack ball from Valverde in midfield, and Chelsea can look to build an attack. Rodrygo is the target of a clever free-kick into the right-hand channel, but Chalobah and Silva combine to crowd the Brazilian out and win possession for Chelsea. [76-90]  FULL-TIME: CHELSEA 0-2 REAL MADRID   Valverde sweeps a pass out to Ceballos, who cuts inside and shoots straight at Kepa from the edge of the area.


In [306]:
ft_comm = temp_df[temp_df['comment_desc'] == 'full time summary']['comment']
full_time_summary = " ".join(ft_comm)
full_time_summary

" We hope you have enjoyed our live coverage of yet another European win for Real Madrid. It's goodbye for now, and we'll see you next time!   While Madrid's Champions League defence continues apace, Chelsea have lost four successive matches since Frank Lampard returned to the club as interim boss. The Blues will now endure just their second trophyless season in the last seven campaigns, and with 17 points separating them from the Premier League's top four, it could be some time before Stamford Bridge hosts another Champions League fixture!   It's all over at Stamford Bridge, and it's another 2-0 win over Chelsea for Real Madrid – Los Blancos take the quarter-final tie 4-0 on aggregate! Chelsea performed admirably for long periods as Cucurella and Kante missed great chances to cut Madrid's lead, but they could not find a response when Rodrygo hammered Vinicius' cut-back in from close range after 58 minutes. A flowing move involving Valverde led to Rodrygo getting a second with 10 minut

# Using BLEU for comaparison

In [311]:
summ_per = summarize_text_nltk(comment)
summ_per

" FULL-TIME: CHELSEA 0-2 REAL MADRID   Valverde sweeps a pass out to Ceballos, who cuts inside and shoots straight at Kepa from the edge of the area. Though Chelsea missed a couple of clear chances at 0-0, Madrid's backline has stood firm for the majority of this game. The Blues have failed to garner any momentum since Rodrygo's goal, and Madrid have actually looked the more likely scorers on the break."

In [314]:
from nltk.translate import bleu

full_time_summ = full_time_summary.split()

sent_b = summ_per.split()


print(bleu(full_time_summ, sent_b))

7.490163752058132e-232


# Using Cosine similarity for comparison

In [277]:
ft_summary = summarize_text_nltk(full_time_summary)
ft_summary

" We hope you have enjoyed our live coverage of yet another European win for Real Madrid. While Madrid's Champions League defence continues apace, Chelsea have lost four successive matches since Frank Lampard returned to the club as interim boss. It's all over at Stamford Bridge, and it's another 2-0 win over Chelsea for Real Madrid – Los Blancos take the quarter-final tie 4-0 on aggregate!"

In [285]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

corpus = [full_time_summary, spacy_summary]

X_train_counts = count_vect.fit_transform(corpus)

pd.DataFrame(X_train_counts.toarray(),columns=count_vect.vocabulary_.keys(),index=['ft_summary_org','ft_summary_crtd'])


,we,hope,you,have,enjoyed,our,live,coverage,of,yet,...,modric,gives,chance,put,is,crowded,near,edge,area,survive
ft_summary_org,1,1,1,1,1,1,1,0,1,5,...,0,1,2,0,2,0,0,1,2,0
ft_summary_crtd,0,0,0,0,0,0,0,1,0,1,...,1,0,0,1,0,1,1,0,0,1


In [286]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

trsfm=vectorizer.fit_transform(corpus)
pd.DataFrame(trsfm.toarray(),columns=vectorizer.vocabulary_.keys(),index=['ft_summary_org','ft_summary_crtd'])


,we,hope,you,have,enjoyed,our,live,coverage,of,yet,...,modric,gives,chance,put,is,crowded,near,edge,area,survive
ft_summary_org,0.059344,0.059344,0.059344,0.059344,0.059344,0.059344,0.059344,0.000000,0.059344,0.211120,...,0.000000,0.059344,0.118689,0.000000,0.118689,0.000000,0.000000,0.059344,0.118689,0.000000
ft_summary_crtd,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.097547,0.000000,0.069405,...,0.097547,0.000000,0.000000,0.097547,0.000000,0.097547,0.097547,0.000000,0.000000,0.097547


In [287]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(trsfm[0:1], trsfm)

array([[1.        , 0.28133489]])